In [1]:
import pandas as pd
import numpy as np

In [2]:
#read in CSVs for EAF., distance from Riparian Buf, forest age/type
path = 'Beyond450_EAF_avg_ExportTable.csv' 
Beyond450_EAF = pd.read_csv(path)
EAF = pd.read_csv("Compartment_avgEAF.csv")
dist_from_RB = pd.read_csv("Compartment_avgDistance_RB.csv")
conversion_1937_1957 = pd.read_csv("conversion_1937_1957.csv")
conversion_1957_1974 = pd.read_csv("conversion_1957_1974.csv")
conversion_1974_1996 = pd.read_csv("conversion_1974_1996.csv")
evergreen = pd.read_csv("evergreen.csv")

In [3]:
# read in CSV for overall inv. supervised data with farmland, riparian features to append former data to
invasion = pd.read_csv("Compartment_Inva_ExportTable.csv")

In [4]:
# get rid of unnecessary columns for ML
df = invasion.iloc[:,[2,5,10,20,21]]
df.head()

,Distance_Riparian_Buffer,Distance_Farmland,HID,overall_in,individual
0,136.620988,0.0,252.0,25-50%,"MIVI-3,LOJA-2,ELUM-1,ROMU-1,LISE-1,CEOR-2,BETH..."
1,-1.000000,0.0,253.0,75-100%,"MIVI-4,CEOR-4,LISE-1,LOJA-1,RUPH-1,ALJU-1"
2,137.086561,0.0,254.0,50-75%,"MIVI-4,CEOR-3,LISE-1,RUPH-1,ELUM-1,LOJA-1,ROMU-1"
3,137.086530,0.0,255.0,50-75%,"MIVI-4,CEOR-3,RUPH-3,LOJA-1,ALJU-1,CIVU-2,LISE-2"
4,137.107700,0.0,256.0,50-75%,"MIVI-3,LISE-2,CEOR-3,RUPH-2,PATO-1,LOJA-2,RUCR..."


In [5]:
# clean data from CSVs, add to df
## start with dist_from_RB 
dist_floodplain = []
EAF_list = []
for i in range(df.shape[0]): #iterate over range of num rows of df
    HID = df.iloc[i,2] #find the compartment ID for current iteration
    idx = dist_from_RB.index[dist_from_RB['HID'] == HID] #find which index of dist_from_RB equals current HID iteration
    mean_distance = dist_from_RB.at[idx[0], 'MEAN_Distance_From_RB'] #retrieves value from mean d col at row specified by idx
    dist_floodplain.append(mean_distance * 3.28) #meters to feet formula

for i in range(df.shape[0]): #same proc for EAF
    HID = df.iloc[i,2] 
    idx = EAF.index[EAF['HID'] == HID] #find which index of overall EAF equals current HID iteration
    mean_EAF = EAF.at[idx[0], 'MEAN_EAF'] #retrieves value from mean d col at row specified by idx
    EAF_list.append(mean_EAF)

In [6]:
# fix dist_floodplain values for PW buffer HIDs
for i in range(df.shape[0]):
    if(df.iloc[i,2] >= 80 and df.iloc[i,2] <= 246):
        dist_floodplain[i] = 300
for i in range(df.shape[0]):
    if(df.iloc[i,2] <= 79):
        dist_floodplain[i] = 0
# assign riparian buffer distance to 300 meters if not within 450 ft of pw buffer
for i in range(df.shape[0]):
    if(df.iloc[i,0] == -1):
        df.iloc[i,0] = 300
# assign new FP dist and EAF lists to df
df = df.assign(EAF = EAF_list)
df = df.assign(Dist_Floodplain = dist_floodplain)
df.head()

,Distance_Riparian_Buffer,Distance_Farmland,HID,overall_in,individual,EAF,Dist_Floodplain
0,136.620988,0.0,252.0,25-50%,"MIVI-3,LOJA-2,ELUM-1,ROMU-1,LISE-1,CEOR-2,BETH...",38.745098,814.578296
1,300.000000,0.0,253.0,75-100%,"MIVI-4,CEOR-4,LISE-1,LOJA-1,RUPH-1,ALJU-1",58.200000,1281.677749
2,137.086561,0.0,254.0,50-75%,"MIVI-4,CEOR-3,LISE-1,RUPH-1,ELUM-1,LOJA-1,ROMU-1",42.064516,898.304107
3,137.086530,0.0,255.0,50-75%,"MIVI-4,CEOR-3,RUPH-3,LOJA-1,ALJU-1,CIVU-2,LISE-2",36.842105,646.795242
4,137.107700,0.0,256.0,50-75%,"MIVI-3,LISE-2,CEOR-3,RUPH-2,PATO-1,LOJA-2,RUCR...",35.789474,578.104646


In [7]:
# add in converted forest
con_37 = [] ## create empty list for each forest conversion
con_57 = []
con_74 = []
for i in range(df.shape[0]):
    HID = df.iloc[i,2]
    if(any(HID == conversion_1937_1957['HID'])): #if the current HID exists in converted forest df
        idx = conversion_1937_1957['HID'] ==  HID
        con_37.append(float(conversion_1937_1957[idx]['PERCENTAGE'])) #extract series from idx in conv forest df then convert to percentage float
    else:
        con_37.append(0) #if tabulate overlay did not overlap with any conv forest, overlay is 0 percent
    if(any(HID == conversion_1957_1974['HID'])): #same procedure for other conv forest dfs
        idx = conversion_1957_1974['HID'] ==  HID
        con_57.append(float(conversion_1957_1974[idx]['PERCENTAGE'])) 
    else:
        con_57.append(0)
    if(any(HID == conversion_1974_1996['HID'])): #same procedure for other conv forest dfs
        idx = conversion_1974_1996['HID'] ==  HID
        con_74.append(float(conversion_1974_1996[idx]['PERCENTAGE'])) 
    else:
        con_74.append(0)
df = df.assign(conversion_1937_1957=con_37) #assign new column to list with percentages
df = df.assign(conversion_1957_1974=con_57)
df = df.assign(conversion_1974_1996=con_74)

In [8]:
#evergreen forest
eg = []
for i in range(df.shape[0]):
    HID = df.iloc[i,2]
    if(any(HID == evergreen['HID'])): #if the current HID exists in converted forest df
        idx = evergreen['HID'] ==  HID
        eg.append(float(evergreen[idx]['PERCENTAGE'])) #extract series from idx in conv forest df then convert to percentage float
    else:
        eg.append(0)
df = df.assign(evergreen_forest = eg)

In [9]:
# percentage ranges to numeric, cover class values
percentage = {'5-25%': 2, '25-50%': 3, '50-75%': 4, '75-100%': 5}
numeric_values = []

for p in df['overall_in']:
    #convert the percentage range to a numeric value
    numeric_value = percentage.get(p)
    numeric_values.append(numeric_value)
    
# add new column with numeric values
df['overall_numeric'] = numeric_values

12

# Remove Outliers

In [10]:
from sklearn.model_selection import train_test_split as tts
from intro_Data_4_3 import *

In [25]:
#Logistic Reg
idx = df["Distance_Riparian_Buffer"] == 300
tempdf = df[idx] #get rid of entries skewed by riparian buf (20 total)
idx = tempdf["Dist_Floodplain"] > 1200 
tempdf = tempdf.drop(tempdf[idx].index) #get rid of outlier far from floodplain 

from sklearn.linear_model import LogisticRegression as LR
from sklearn.preprocessing import StandardScaler as SS
from sklearn.decomposition import PCA

X = np.array(tempdf.drop(columns=['individual', 'overall_in','overall_numeric', 'HID']))
y = np.array(tempdf['overall_numeric'])

Xtrain,Xtest,ytrain,ytest = tts(X,y,test_size=0.4, random_state = 146) #test size of 0.2 yielded better results, but K-fold with k=5 did not

ss = SS()
Xtrain = ss.fit_transform(Xtrain)
Xtest = ss.transform(Xtest)
# PC = PCA(n_components = 2)
# Xtrain = PC.fit_transform(Xtrain)
# Xtest = PC.transform(Xtest)

log = LR()
log.fit(Xtrain,ytrain)
y_pred = log.predict(Xtest)
print(log.score(Xtest, ytest))

matrix = compare_classes(y_pred, ytest)
matrix

#wildly inconsistent, but scaling + PCs make it more accurate (occasionally)

0.6428571428571429
Test accuracy = 0.64


(Predicted  2  3  4  5
 Actual               
 2          2  0  0  0
 3          2  5  1  0
 4          0  0  2  2,
 0.6428571428571429)

In [29]:
# RFC
from sklearn.ensemble import RandomForestClassifier as RFC

X = np.array(tempdf.drop(columns=['individual', 'overall_in','overall_numeric', 'HID']))
y = np.array(tempdf['overall_numeric'])

Xtrain,Xtest,ytrain,ytest = tts(X,y,test_size=0.4) #test size of 0.2 yielded better results, but K-fold with k=5 did not

# do not scale for ensemble

# PC = PCA()
# Xtrain = PC.fit_transform(Xtrain)
# Xtest = PC.transform(Xtest)

#random forests can handle high dimensionality

rfc = RFC(random_state = 14) #reducing n_estimators from 100 to 50 increased accuracy
rfc.fit(Xtrain,ytrain)
y_pred = rfc.predict(Xtest)
print("Random Forest Accuracy", rfc.score(Xtest, ytest))

matrix = compare_classes(y_pred, ytest)
matrix

Random Forest Accuracy 0.5
Test accuracy = 0.50


(Predicted  2  3  4  5
 Actual               
 2          0  1  0  0
 3          1  6  2  0
 4          0  1  1  2,
 0.5)

In [42]:
#Neural Network (full dataset)
from sklearn.neural_network import MLPClassifier

# Assuming you have already loaded your data into 'tempdf'

X = np.array(df.drop(columns=['individual', 'overall_in', 'overall_numeric', 'HID']))
y = np.array(df['overall_numeric'])

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.4)

ss = SS()
Xtrain = ss.fit_transform(Xtrain)
Xtest = ss.transform(Xtest)
# PC = PCA(n_components = 3)
# Xtrain = PC.fit_transform(Xtrain)
# Xtest = PC.transform(Xtest)

# Create and train the MLPClassifier
mlp = MLPClassifier(random_state=42, max_iter = 400)
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Calculate accuracy
accuracy = mlp.score(X_test, y_test)
print("MLP Classifier Accuracy:", accuracy)

# You can also use your compare_classes function to display the confusion matrix
matrix = compare_classes(y_pred, y_test)
matrix
#LESS ACCURATE

MLP Classifier Accuracy: 0.34782608695652173
Test accuracy = 0.35


(Predicted  2  3  4  5
 Actual               
 2          0  2  0  0
 3          3  7  4  0
 4          0  3  1  1
 5          0  0  2  0,
 0.34782608695652173)

In [80]:
#Neural Network (no outliers)
from sklearn.neural_network import MLPClassifier

X = np.array(tempdf.drop(columns=['individual', 'overall_in', 'overall_numeric', 'HID']))
y = np.array(tempdf['overall_numeric'])

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.4)

ss = SS()
Xtrain = ss.fit_transform(Xtrain)
Xtest = ss.transform(Xtest)
# PC = PCA(n_components = 3)
# Xtrain = PC.fit_transform(Xtrain)
# Xtest = PC.transform(Xtest)

# Create and train the MLPClassifier
mlp = MLPClassifier(random_state=14, max_iter = 300)
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Calculate accuracy
accuracy = mlp.score(X_test, y_test)
print("MLP Classifier Accuracy:", accuracy)

# You can also use your compare_classes function to display the confusion matrix
matrix = compare_classes(y_pred, y_test)
matrix


MLP Classifier Accuracy: 0.5714285714285714
Test accuracy = 0.57


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


(Predicted  2  3  4  5
 Actual               
 2          1  2  0  0
 3          2  4  0  0
 4          0  1  2  0
 5          0  0  1  1,
 0.5714285714285714)

In [62]:
#RFC determine best hyperparams
##min_samples_split
min_samples_range = np.arange(2, 11)
max_depth_range = np.arange(2, 11)
columns = ['min_samples_range', 'max_depth_range', 'accuracy']
scores = pd.DataFrame(columns = columns)
for s in min_samples_range:
    for m in max_depth_range:
        rfc = RFC(min_samples_split = s, max_depth = m)
        train_scores,test_scores = do_Kfold(rfc,X,y,5)
        new_row = {'min_samples_range': s,'max_depth_range': m, 'accuracy': np.mean(np.array(test_scores))}
        scores = pd.concat([scores, pd.DataFrame(new_row, index=[0])], ignore_index=True)



In [64]:
#max value for scores generated above
# min_samples_split commonly delivers best accuracy at 2

# min_samples_range           3
# max_depth_range             2
# accuracy             0.484848
# Name: 9, dtype: object

# min_samples_range           3
# max_depth_range             5
# accuracy             0.514286

# min_samples_range           3
# max_depth_range             9
# accuracy             0.514286
scores.loc[scores['accuracy'].idxmax(),:]
# np.mean(np.array(scores['accuracy']))

0.42010582010582004

In [65]:
#MLP classifier determine best hyperparams
# Define the ranges for hyperparameters
hidden_layer_sizes_range = [(10,), (20,), (30,)]
activation_range = ['relu', 'logistic']
alpha_range = [0.0001, 0.001, 0.01]
columns = ['hidden_layer_sizes', 'activation', 'alpha', 'accuracy']
scores = pd.DataFrame(columns=columns)

for hidden_layer_sizes in hidden_layer_sizes_range:
    for activation in activation_range:
        for alpha in alpha_range:
            mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, alpha=alpha, max_iter = 10000)
            train_scores, test_scores = do_Kfold(mlp, X, y, 5)
            new_row = {
                'hidden_layer_sizes': hidden_layer_sizes,
                'activation': activation,
                'alpha': alpha,
                'accuracy': np.mean(np.array(test_scores))
            }
            scores = pd.concat([scores, pd.DataFrame(new_row, index=[0])], ignore_index=True)

# Now 'scores' DataFrame contains accuracy scores for different hyperparameter combinations


In [68]:
#max value for scores generated above
# min_samples_split commonly delivers best accuracy at 2

scores.loc[scores['accuracy'].idxmax(),:]
#10
#relu
#0.01
#0.6

#20
#logistic
#0.0001
#0.62

# hidden_layer_sizes          10
# activation            logistic
# alpha                   0.0001
# accuracy              0.628571

# hidden_layer_sizes          10
# activation            logistic
# alpha                   0.0001
# accuracy              0.485714
# Name: 3, dtype: object

# hidden_layer_sizes          10
# activation            logistic
# alpha                    0.001
# accuracy                   0.6

# np.mean(np.array(scores['accuracy']))

hidden_layer_sizes          20
activation            logistic
alpha                     0.01
accuracy                   0.6
Name: 11, dtype: object

# Algorithm consistently shows max accuracy with
    hidden_layer_sizes          10-20
    activation - logistic
    alpha: 0.0001-0.001

## Fine-tune further

In [69]:
#MLP classifier determine best hyperparams pt. 2

# Define the ranges for hyperparameters
hidden_layer_sizes_range = np.arange(10,16) #use arange to create list from 10 to 20
activation_range = 'logistic'
alpha_range = np.arange(1,6) * 0.0001
columns = ['hidden_layer_sizes', 'alpha', 'accuracy']
scores = pd.DataFrame(columns=columns)

for hidden_layer_sizes in hidden_layer_sizes_range:
    for alpha in alpha_range:
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation='logistic', alpha=alpha, max_iter = 10000)
        train_scores, test_scores = do_Kfold(mlp, X, y, 5)
        new_row = {
            'hidden_layer_sizes': hidden_layer_sizes,
            'alpha': alpha,
            'accuracy': np.mean(np.array(test_scores))
        }
        scores = pd.concat([scores, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [72]:
scores.loc[scores['accuracy'].idxmax(),:]

# hidden_layer_sizes        10
# alpha                 0.0004
# accuracy                 0.6
# Name: 3, dtype: object

# hidden_layer_sizes          10
# alpha                   0.0003
# accuracy              0.628571

# hidden_layer_sizes          13
# alpha                   0.0004
# accuracy              0.628571

# hidden_layer_sizes          15
# alpha                   0.0003
# accuracy              0.685714

# hidden_layer_sizes          11
# alpha                   0.0005
# accuracy              0.628571

# hidden_layer_sizes          15
# alpha                   0.0005
# accuracy              0.685714
#np.mean(np.array(scores['accuracy']))

hidden_layer_sizes          15
alpha                   0.0005
accuracy              0.685714
Name: 29, dtype: object

In [107]:
#try MLP with new params KFold

X = np.array(tempdf.drop(columns=['individual', 'overall_in', 'overall_numeric', 'HID']))
y = np.array(tempdf['overall_numeric'])

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.4, random_state = 146)

# ss = SS()
# Xtrain = ss.fit_transform(Xtrain)
# Xtest = ss.transform(Xtest)
# PC = PCA(n_components = 3)
# Xtrain = PC.fit_transform(Xtrain)
# Xtest = PC.transform(Xtest)

# Create and train the MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=10, activation='logistic', alpha=0.0004, max_iter = 10000, random_state= 146)
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Calculate accuracy
accuracy = mlp.score(X_test, y_test)
print("MLP Classifier Accuracy:", accuracy)

# You can also use your compare_classes function to display the confusion matrix
matrix = compare_classes(y_pred, y_test)
matrix

MLP Classifier Accuracy: 0.7142857142857143
Test accuracy = 0.71


(Predicted  2  3  4  5
 Actual               
 2          2  0  0  0
 3          2  5  1  0
 4          0  0  2  1
 5          0  0  0  1,
 0.7142857142857143)

In [93]:
from sklearn.model_selection import cross_val_score

# Create the random forest classifier
rfc = RFC(random_state=146, n_estimators=100, max_depth = 8, min_samples_split = 5)

#convert to numpy array for Kfold function
X_arr = np.array(X)
y_arr = np.array(y)

# Perform 5-fold cross-validation
train_scores,test_scores = do_Kfold(rfc,X_arr,y_arr,5, random_state = 146)

# Print the cross-validation scores
print("Cross-Validation Scores:", test_scores)

# Calculate and print the mean and standard deviation of the scores
print("Mean Accuracy:", np.mean(np.array(test_scores)))
print("Standard Deviation of Test Accuracy:", np.array(test_scores).std())

Cross-Validation Scores: [0.42857142857142855, 0.42857142857142855, 0.42857142857142855, 0.5714285714285714, 0.42857142857142855]
Mean Accuracy: 0.45714285714285713
Standard Deviation of Test Accuracy: 0.05714285714285714


In [109]:
tempdf.head()

,Distance_Riparian_Buffer,Distance_Farmland,HID,overall_in,individual,EAF,Dist_Floodplain,conversion_1937_1957,conversion_1957_1974,conversion_1974_1996,evergreen_forest,overall_numeric
6,300.0,0.000000,258.0,25-50%,"MIVI-2,LISE-1,ROMU-1,CAAC-2,LOJA-2,CEOR-3",32.571429,818.178278,0.0,0.0,99.999877,0.000000,3
8,300.0,173.037653,264.0,25-50%,"RUPH-2,MIVI-3,LOJA-1,PATO-2,CEOR-1,ROMU-1,LISE-1",64.142857,675.009333,0.0,0.0,0.000000,0.000000,3
9,300.0,361.244783,265.0,5-25%,"CEOR-2,MIVI-2,LISE-1,RUPH-1",45.309091,698.781851,0.0,0.0,0.000000,0.000000,2
10,300.0,522.854292,266.0,5-25%,"CEOR-2,MIVI-2,LISE-1,RUPH-1",43.714286,580.959644,0.0,0.0,0.000000,4.782920,2
11,300.0,331.102949,267.0,5-25%,"RUPH-1,CEOR-2,LOJA-1",61.739130,538.781197,0.0,0.0,0.000000,78.577919,2
